In [1]:
using JuMP
import DataFrames
import CPLEX
import Plots
import StatsPlots

In [14]:
function ThermalGenerator(
    min::Float64,
    max::Float64,
    fixed_cost::Float64,
    variable_cost::Float64,
)
    return (
        min = min,
        max = max,
        fixed_cost = fixed_cost,
        variable_cost = variable_cost,
    )
end

generators = [
    ThermalGenerator(0.0, 37337.0, 0.0, 50.0),
    ThermalGenerator(0.0, 23250.0, 0.0, 100.0),
    ThermalGenerator(0.0, 41201.0, 0.0, 50.0),
]

3-element Vector{NamedTuple{(:min, :max, :fixed_cost, :variable_cost), NTuple{4, Float64}}}:
 (min = 0.0, max = 37337.0, fixed_cost = 0.0, variable_cost = 50.0)
 (min = 0.0, max = 23250.0, fixed_cost = 0.0, variable_cost = 100.0)
 (min = 0.0, max = 41201.0, fixed_cost = 0.0, variable_cost = 50.0)

In [15]:
WindGenerator(variable_cost::Float64) = (variable_cost = variable_cost,)

wind_generator = WindGenerator(25.0)

(variable_cost = 25.0,)

In [16]:
function Scenario(demand::Float64, wind::Float64)
    return (demand = demand, wind = wind)
end

scenario = Scenario(128089.0,26301.0)

(demand = 128089.0, wind = 26301.0)

In [17]:
function solve_economic_dispatch(generators::Vector, wind, scenario)
    # Define the economic dispatch (ED) model
    model = Model(CPLEX.Optimizer)
    set_silent(model)
    # Define decision variables
    # power output of generators
    N = length(generators)
    @variable(model, generators[i].min <= g[i = 1:N] <= generators[i].max)
    # wind power injection
    @variable(model, 0 <= w <= scenario.wind)
    # Define the objective function
    @objective(
        model,
        Min,
        sum(generators[i].variable_cost * g[i] for i in 1:N) +
        wind.variable_cost * w,
    )
    # Define the power balance constraint
    @constraint(model, sum(g[i] for i in 1:N) + w == scenario.demand)
    # Solve statement
    optimize!(model)
    # return the optimal value of the objective function and its minimizers
    return (
        g = value.(g),
        w = value(w),
        wind_spill = scenario.wind - value(w),
        total_cost = objective_value(model),
    )
end

solve_economic_dispatch (generic function with 1 method)

In [18]:
length(generators)

3

In [19]:
solution = solve_economic_dispatch(generators, wind_generator, scenario);

println("Dispatch of Generators: ", solution.g, " MW")
println("Dispatch of Wind: ", solution.w, " MW")
println("Wind spillage: ", solution.wind_spill, " MW")
println("Total cost: \$", solution.total_cost)

Dispatch of Generators: [37337.0, 23250.0, 41201.0] MW
Dispatch of Wind: 26301.0 MW
Wind spillage: 0.0 MW
Total cost: $6.909425e6


In [8]:
function scale_generator_cost(g, scale)
    return ThermalGenerator(g.min, g.max, g.fixed_cost, scale * g.variable_cost)
end

scale_generator_cost (generic function with 1 method)

In [22]:
start = time()
c_g_scale_df = DataFrames.DataFrame(;
    # Scale factor
    scale = Float64[],
    # Dispatch of Generator 1 [MW]
    dispatch_G1 = Float64[],
    # Dispatch of Generator 2 [MW]
    dispatch_G2 = Float64[],
    # Dispatch of Generator 3 [MW]
    dispatch_G3 = Float64[],
    # Dispatch of Wind [MW]
    dispatch_wind = Float64[],
    # Spillage of Wind [MW]
    spillage_wind = Float64[],
    # Total cost [$]
    total_cost = Float64[],
)

Row,scale,dispatch_G1,dispatch_G2,dispatch_G3,dispatch_wind,spillage_wind,total_cost
,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [21]:
for c_g1_scale in 0.5:0.1:0.7
    # Update the incremental cost of the first generator at every iteration.
    new_generators = scale_generator_cost.(generators, [c_g1_scale, 1.0])
    # Solve the economic-dispatch problem with the updated incremental cost
    sol = solve_economic_dispatch(new_generators, wind_generator, scenario)
    push!(
        c_g_scale_df,
        (c_g1_scale, sol.g[1], sol.g[2], sol.w, sol.wind_spill, sol.total_cost),
    )
end

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 3 and 2

In [11]:
print(string("elapsed time: ", time() - start, " seconds"))

elapsed time: 18.258000135421753 seconds